<a href="https://colab.research.google.com/github/dvschultz/ai/blob/master/StyleGAN2_Colab_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training StyleGAN2 on Colab
Y’all won’t stop asking me about this so here ya go 😂

If it were me I’d sign up for Colab Pro ($10/month) to get a couple extra hours of training time in per session. But you crazy kids can do whatever you want.

##Mounting Google Drive
So I’m actually gonna install my entire repo directly into Google Drive. This will make the setup a little easier, but its a little strange I admit.

First, connect your Drive to Colab.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##Install the repo
**Only do this for the first time ever setting this up!**

If this is your first time ever running this notebook, you’ll want to install my fork of StyleGAN2 to your Drive account. Make sure you have ample space on your Drive (I’d say at least 50GB). This will install the repo and add some dependecies (like transferring from FFHQ the first time).

Every time after your first use of this notebook you’ll want to skip this cell and run the cell after this.

In [0]:
#SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/My\ Drive/
!mkdir stylegan2-colab
%cd stylegan2-colab/
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
%mkdir datasets

/content/drive/My Drive
/content/drive/My Drive/stylegan2-colab-test
Cloning into 'stylegan2'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 261 (delta 0), reused 1 (delta 0), pack-reused 257
Receiving objects: 100% (261/261), 15.25 MiB | 14.95 MiB/s, done.
Resolving deltas: 100% (137/137), done.
/content/drive/My Drive/stylegan2-colab-test/stylegan2
/content/drive/My Drive/stylegan2-colab-test/stylegan2/pkl
Downloading...
From: https://drive.google.com/uc?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
To: /content/drive/My Drive/stylegan2-colab-test/stylegan2/pkl/inception_v3_features.pkl
87.3MB [00:01, 68.7MB/s]
/content/drive/My Drive/stylegan2-colab-test/stylegan2
/content/drive/My Drive/stylegan2-colab-test/stylegan2/results/00001-pretrained
Downloading...
From: https://drive.google.com/uc?id=1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
To: /content/drive/My Drive/stylegan2-colab-test/stylegan2/results/0

##Picking up from a previous session
If you already have the StyleGAN2 repo installed in Drive skip the above cell and run the following. This will make sure you have the latest version in case I make updates.

In [0]:
#USE this if you already have a stylegan2 folder in google drive
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!git pull

HEAD is now at cdc8c0e set latest


##Make sure Tensorflow 1.15 is set
Colab now defaults to Tensorflow 2. Make sure you run this cell to reset it to TF1.

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


##Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). 

I’ve seen some recommendations to run this command every time you restart your Colab machine. I think if you ahve a small-ish dataset (< 2000 images) that’s probably unnecessary.

I recommend you upload your dataset to Google Drive and copy its path from the Drive folder in Colab and paste its path in the below cell.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [0]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
!python dataset_tool.py create_from_images ./datasets/birdaus /content/drive/My\ Drive/Datasets/birdsaus/sq-1024

Loading images from "/content/drive/My Drive/Datasets/birdsaus/sq-1024"
100% 597/597 [06:33<00:00,  1.52it/s]


##Training
Note: this will require you to restart your Colab machine every 8–16 hours. You’ve been warned!

This library is set up to automatically find your latest .pkl file so it should keep re-training and making progress.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.

`--metrics`

METRICS DON’T MATTER. It’s art! Use your eyes. Set `--metrics=None` and live your life.

If you must use metrics, you have a few options. `fid50k`, the default, uses Frechet Inception Distance score. It’s what was used in StyleGAN1 and what most people know. It’s fine for images of animals and things, but it’s not great. `ppl_wend` is what StyleGAN2 prefers and claims to be more accurate. There are a bunch of other options but I’d recommend you stick with those. Note that both of these take 30–45minutes to run every time it runs so that cuts into your training time in Colab.


In [0]:
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=birdaus --mirror-augment=true --metrics=None

Local submit - run_dir: results/00002-stylegan2-birdaus-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x76da000 @  0x7f3057e80001 0x7f305490c765 0x7f3054970dc0 0x7f3054972c5f 0x7f3054a09238 0x50ac25 0x50d390 0x508245 0x509642 0x595311 0x54a6ff 0x551b81 0x5a067e 0x50d966 0x508245 0x58958c 0x5a067e 0x50d966 0x508245 0x58958c 0x5a067e 0x50d966 0x509d48 0x50aa7d 0x50c5b9 0x509d48 0x50aa7d 0x50c5b9 0x508245 0x58958c 0x5a067e
tcmalloc: large alloc 4294967296 bytes == 0x7f2e8e2c6000 @  0x7f3057e7e1e7 0x7f305490c5e1 0x7f3054970e88 0x7f3054971147 0x7f3054a09118 0x50ac25 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x509d48 0x50aa7d 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245
tcmalloc: large alloc 4294967296 bytes == 0x7f2d8d2c4000 @  0x7f3057e

Once running, your training files will show up in the results folder.

#Testing the model (generating images)
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [0]:
!python run_generator.py generate-images --network=/content/ladiesfloralcrop-network-snapshot-010237.pkl --seeds=3875451-3876000 --truncation-psi=0.7

Local submit - run_dir: results/00000-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/ladiesfloralcrop-network-snapshot-010237.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.
Generating image for seed 1 (0/25) ...
Generating image for seed 2 (1/25) ...
Generating image for seed 3 (2/25) ...
Generating image for seed 4 (3/25) ...
Generating image for seed 5 (4/25) ...
Generating image for seed 6 (5/25) ...
Generating image for seed 7 (6/25) ...
Generating image for seed 8 (7/25) ...
Generating image for seed 9 (8/25) ...
Generating image for seed 10 (9/25) ...
Generating image for seed 11 (10/25) ...
Generating image for seed 12 (11/25) ...
Generating image for seed 13 (12/25) ...
Generating image for seed 14 (13/25) ...
Generating image for seed 15 (14/25) ...
Generating imag

Let’s zip the generated files and download them.

In [0]:
!zip -r generated-0.7.zip /content/stylegan2/results/00000-generate-images

  adding: content/stylegan2/results/00000-generate-images/ (stored 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0025.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0014.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0007.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0018.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0017.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0010.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0015.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0024.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0021.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0004.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/log.txt (deflated 74%

##Interpolation


In [0]:
!python run_generator.py generate-latent-walk --network=/content/ladiesfloralcrop-network-snapshot-010237.pkl --seeds=3,11,17,25,3 --frames 200 --truncation-psi=0.7

Local submit - run_dir: results/00002-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "/content/ladiesfloralcrop-network-snapshot-010237.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for step 0/204 ...
Generating image for step 1/204 ...
Generating image for step 2/204 ...
Generating image for step 3/204 ...
Generating image for step 4/204 ...
Generating image for step 5/204 ...
Generating image for step 6/204 ...
Generating image for step 7/204 ...
Generating image for step 8/204 ...
Generating image for step 9/204 ...
Generating image for step 10/204 ...
Generating image for step 11/204 ...
Generating image for step 12/204 ...
Generating image for step 13/204 ...
Generating image for step 14/204 ...
Generating image for step 15/204 ...
Generating image for step 16/204 ...
Generati

In [0]:
#convert to video 
!ffmpeg -r 24 -i ./results/00001-generate-latent-walk/step%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
rm -r /content/drive/My Drive/stylegan2-colab-test/stylegan2/results/00002-stylegan2-birdaus-1gpu-config-f

rm: cannot remove '/content/drive/My': No such file or directory
rm: cannot remove 'Drive/stylegan2-colab-test/stylegan2/results/00002-stylegan2-birdaus-1gpu-config-f': No such file or directory
